# This notebook will be used for the entire Week 3 assignment. Each part will be noted.

## <b> Week 3 Assignment - Part 1 </b>

In [78]:
#Importing pandas and numpy libraries
import pandas as pd
import numpy as np

In [80]:
#Importing request command from urllib for scraping the Wikipedia link
import urllib.request 

In [81]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [82]:
from urllib.request import urlopen

In [83]:
#Importing BeautifulSoup library
conda install -c anaconda beautifulsoup4

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - beautifulsoup4


The following packages will be UPDATED:

  openssl            conda-forge::openssl-1.1.1f-h516909a_0 --> anaconda::openssl-1.1.1-h7b6447c_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates    conda-forge::ca-certificates-2020.4.5~ --> anaconda::ca-certificates-2020.1.1-0
  certifi            conda-forge::certifi-2020.4.5.1-py36h~ --> anaconda::certifi-2020.4.5.1-py36_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [84]:
from bs4 import BeautifulSoup

In [85]:
#Setting an indicator for the Wikipedia link
link = urlopen(url)

In [86]:
#Convert the link's content according to the BeautifulSoup interface
soup = BeautifulSoup(link, "html")

In [87]:
#Getting the table from the link
postTable = soup.table

In [88]:
#A list for storing the postal codes information
cells = []
#Flag for skipping the first empty row of the table
flag = 0
for row in postTable.findAll('tr'):
    #Going over code lines who represents each of the cells of the current row in the loop and converting them into a string format
    cell = list(map(str,row.findAll('td')))
    #Cleaning the data from the code parts and replacing the slashes into commas at the 3rd cell
    ctext = [c.replace('<td>','').replace('\n</td>','').replace('\n</pre>','').replace('\n<pre>','').replace('amp;','').replace(' /',',') for c in cell]
    if flag == 1:
        #Filtering the not assigned postal codes
        if ctext[1]!='Not assigned': 
            cells.append(ctext)
    else:
        flag = 1

#Creating the database
torfd = pd.DataFrame(cells,columns = ['PostalCode','Borough','Neighborhood'])
torfd

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [89]:
torfd.shape #shape of the dataframe

(103, 3)

## <b> Week 3 Assignment - Part 2 </b>

In [90]:
#Getting the csv data for the neighborhoods' lat-lon coordinates and converting it into a dataframe
lltor = pd.read_csv('http://cocl.us/Geospatial_data')

In [91]:
#Modifying the dataframe by setting the Postal Code as index for the columns and the lat-lon as indices for the rows.
l2 = lltor.set_index('Postal Code').transpose()
l2.head()

Postal Code,M1B,M1C,M1E,M1G,M1H,M1J,M1K,M1L,M1M,M1N,...,M9A,M9B,M9C,M9L,M9M,M9N,M9P,M9R,M9V,M9W
Latitude,43.806686,43.784535,43.763573,43.770992,43.773136,43.744734,43.727929,43.711112,43.716316,43.692657,...,43.667856,43.650943,43.643515,43.756303,43.724766,43.706876,43.696319,43.688905,43.739416,43.706748
Longitude,-79.194353,-79.160497,-79.188711,-79.216917,-79.239476,-79.239476,-79.262029,-79.284577,-79.239476,-79.264848,...,-79.532242,-79.554724,-79.577201,-79.565963,-79.532242,-79.518188,-79.532242,-79.554724,-79.588437,-79.594054


In [92]:
#Arranging the lat-lon data according to the order of the postal codes in torfd dataframe
latforTorfd = []
lonforTorfd = []
for pct in torfd['PostalCode']:
    latforTorfd.append(l2[pct]['Latitude'])
    lonforTorfd.append(l2[pct]['Longitude'])


In [93]:
#Adding new columns of lat and lon to torfd
torfd['Latitude'] = latforTorfd
torfd['Longitude'] = lonforTorfd

In [94]:
torfd

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road , Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## <b> Week 3 Assignment - Part 3 </b>

In [95]:
torfd['Borough'].unique().shape[0] #Setting the num of clusters as the num of Toronto's boroughs

10

In [56]:
from sklearn.cluster import KMeans #Importing the kmeans model from scikit-learn library
import folium #Importing Folium

In [59]:
#Converting the lat-lon data into an array for the KMeans model
lldata = torfd[['Latitude','Longitude']]
lldata = np.asarray(lldata)

In [61]:
#Performing the KMeans model and retreiving the labels
num_clusters = 10 

kmModel = KMeans(init="k-means++", n_clusters=num_clusters, n_init=12)
kmModel.fit(lldata)
labels = kmModel.labels_
print(labels)

[8 4 3 0 3 1 9 8 4 3 0 1 9 4 4 3 0 1 9 4 3 0 9 4 3 3 5 8 0 4 3 6 5 8 7 4 3
 3 5 8 0 4 3 3 5 8 7 4 3 7 7 5 8 7 4 0 6 7 5 8 7 0 0 6 7 5 8 0 0 6 7 5 0 0
 3 6 1 1 2 0 3 6 2 3 3 2 3 3 6 7 2 3 3 1 1 9 3 3 6 3 4 6 6]


In [62]:
torfd['Labels'] = labels #Adding the labels of the clusters as a new column for torfd

In [63]:
torfd

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Labels
0,M3A,North York,Parkwoods,43.753259,-79.329656,8
1,M4A,North York,Victoria Village,43.725882,-79.315572,4
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,3
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,0
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3
...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road , Old Mill North",43.653654,-79.506944,6
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,3
100,M7Y,East Toronto,Business reply mail Processing CentrE,43.662744,-79.321558,4
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,6


In [67]:
#Installing Geocoder for retreiving the lat=l
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

address = 'Toronto'

geolocator = Nominatim(user_agent="tr_explorer")
location = geolocator.geocode(address)
torlat = location.latitude
torlon = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(torlat, torlon))

Solving environment: done

# All requested packages already installed.

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [77]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[torlat, torlon], zoom_start=10)

# set color scheme for the clusters
colors_array = cm.rainbow(np.linspace(0, 1, num_clusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
for lat, lon, posCod, neigh, cluster in zip(torfd['Latitude'], torfd['Longitude'], torfd['PostalCode'], torfd['Neighborhood'], torfd['Labels']):
    label = folium.Popup(str(posCod) +': '+str(neigh)+ '. Cluster ' + str(cluster+1), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters